In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

## Recopilación de datos

In [ ]:
data =  pd.read_csv("/workspaces/Gupir11-machine-learning/data/raw/bank-marketing-campaign-data.csv",sep=';')
data.head()